# MetaPop: Metagenomic Population Diversity Analysis

This notebook provides an interactive interface for running MetaPop on Google Colab.

**MetaPop** analyzes:
- **Macrodiversity**: Sample-level diversity (richness, Shannon's H, Simpson, etc.)
- **Microdiversity**: Within-population genetic variation (pi, theta, Tajima's D, pN/pS)

---

## 1. Setup and Installation

Run this cell first to install all required dependencies.

In [ ]:
# Step 1: Install system dependencies (samtools, bcftools, prodigal)
!apt-get update -qq 2>/dev/null
!apt-get install -y -qq samtools bcftools prodigal 2>/dev/null

# Step 2: Install Python dependencies
!pip install -q numpy pandas scipy pysam matplotlib seaborn scikit-learn plotly ipywidgets

# Step 3: Clone and install MetaPop
import os
if not os.path.exists('metapop'):
    !git clone https://github.com/espickle1/metapop.git

%cd metapop
!pip install -q .
%cd ..

print("Base dependencies and MetaPop installed!")

## 2. Mount Google Drive

Mount your Google Drive to access data files and save results.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Your files will be available at /content/drive/MyDrive/

In [ ]:
# Process BAM files from Google Drive to local Colab storage
import shutil
import os

# Define paths
drive_bam_dir = '/content/drive/MyDrive/metapop_data/bams'  # Your Drive path
local_bam_dir = '/content/local_bams'  # Local Colab storage

# Create local directory
os.makedirs(local_bam_dir, exist_ok=True)

# Copy all BAM files to local storage
print("Copying BAM files to local storage (this may take a while)...")
for f in os.listdir(drive_bam_dir):
    if f.endswith('.bam') or f.endswith('.bai') or f.endswith('.bam.bai'):
        src = os.path.join(drive_bam_dir, f)
        dst = os.path.join(local_bam_dir, f)
        if not os.path.exists(dst):
            print(f"  Copying {f}...")
            shutil.copy2(src, dst)
        else:
            print(f"  {f} already exists locally")

# Index BAM files if index doesn't exist
print("\nChecking/creating BAM indexes...")
for f in os.listdir(local_bam_dir):
    if f.endswith('.bam'):
        bam_path = os.path.join(local_bam_dir, f)
        bai_path = bam_path + '.bai'
        alt_bai_path = bam_path[:-4] + '.bai'  # file.bai instead of file.bam.bai
        if not os.path.exists(bai_path) and not os.path.exists(alt_bai_path):
            print(f"  Indexing {f}...")
            os.system(f'samtools index "{bam_path}"')
        else:
            print(f"  {f} already indexed")

print(f"\nDone! Use '{local_bam_dir}' as your BAM Directory in the widget above.")

# Verify BAM files are readable
print("\nVerifying BAM files...")
!ls -la {local_bam_dir}/*.bam* | head -10

In [ ]:
# Download files from shared Google Drive links
!pip install -q gdown
import os
import re
import ipywidgets as widgets
from IPython.display import display, HTML

# --- Input widgets ---
display(HTML('''
<style>
.drive-input textarea, .drive-input input {
    color: #000 !important; font-family: monospace; font-size: 13px;
}
</style>
'''))

print("=== Download Files from Google Drive Links ===")
print("Paste shared Google Drive links below, then click 'Download All'.")
print("Link format: https://drive.google.com/file/d/FILE_ID/view?...")
print()

bam_links = widgets.Textarea(
    value='',
    placeholder='Paste one Google Drive link per line, e.g.:\n'
                'https://drive.google.com/file/d/ABC123.../view?usp=drive_link\n'
                'https://drive.google.com/file/d/DEF456.../view?usp=drive_link',
    description='BAM files:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='95%', height='120px')
)
bam_links.add_class('drive-input')

ref_link = widgets.Text(
    value='',
    placeholder='https://drive.google.com/file/d/FILE_ID/view?usp=drive_link',
    description='Reference FASTA:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='95%')
)
ref_link.add_class('drive-input')

download_output = widgets.Output(layout={'border': '1px solid #ccc',
                                         'min_height': '60px',
                                         'max_height': '400px',
                                         'overflow_y': 'auto',
                                         'padding': '8px'})

def extract_file_id(url):
    """Extract Google Drive file ID from a shared link."""
    url = url.strip()
    # Match /d/FILE_ID/ or id=FILE_ID
    m = re.search(r'/d/([a-zA-Z0-9_-]+)', url)
    if m:
        return m.group(1)
    m = re.search(r'id=([a-zA-Z0-9_-]+)', url)
    if m:
        return m.group(1)
    return None

def download_files(btn):
    """Download all files from the provided links."""
    btn.disabled = True
    btn.description = 'Downloading...'
    download_output.clear_output()

    with download_output:
        bam_dir = '/content/local_bams'
        ref_dir = '/content/local_refs'
        os.makedirs(bam_dir, exist_ok=True)
        os.makedirs(ref_dir, exist_ok=True)

        # --- Download BAM files ---
        bam_lines = [l.strip() for l in bam_links.value.strip().split('\n') if l.strip()]
        if bam_lines:
            print(f"{'='*50}")
            print(f"Downloading {len(bam_lines)} BAM file(s)...")
            print(f"{'='*50}")
            for i, link in enumerate(bam_lines, 1):
                fid = extract_file_id(link)
                if not fid:
                    print(f"\n  [{i}] Could not extract file ID from: {link[:80]}")
                    continue
                # gdown will detect the filename from Drive
                print(f"\n  [{i}] File ID: {fid}")
                dl_url = f"https://drive.google.com/uc?id={fid}"
                ret = os.system(f'gdown "{dl_url}" -O /tmp/gdown_temp 2>&1')
                if ret == 0 and os.path.exists('/tmp/gdown_temp'):
                    # Try to get the real filename from gdown
                    # Re-download with --fuzzy to get filename
                    os.remove('/tmp/gdown_temp')
                    ret = os.system(f'cd "{bam_dir}" && gdown "{dl_url}" 2>&1')
                    if ret == 0:
                        print(f"      Saved to {bam_dir}/")
                    else:
                        print(f"      Download failed (exit code {ret})")
                else:
                    # Try direct with --fuzzy flag
                    ret = os.system(f'cd "{bam_dir}" && gdown --fuzzy "{link}" 2>&1')
                    if ret == 0:
                        print(f"      Saved to {bam_dir}/")
                    else:
                        print(f"      Download failed")
        else:
            print("No BAM links provided (skipping).")

        # --- Download reference FASTA ---
        ref_url = ref_link.value.strip()
        if ref_url:
            print(f"\n{'='*50}")
            print("Downloading reference FASTA...")
            print(f"{'='*50}")
            fid = extract_file_id(ref_url)
            if fid:
                ret = os.system(
                    f'cd "{ref_dir}" && gdown "https://drive.google.com/uc?id={fid}" 2>&1'
                )
                if ret == 0:
                    # Check what was downloaded
                    files = os.listdir(ref_dir)
                    for fn in files:
                        fpath = os.path.join(ref_dir, fn)
                        fsize = os.path.getsize(fpath)
                        print(f"  Saved: {fn} ({fsize:,} bytes)")
                        # Sanity check
                        with open(fpath, 'rb') as f:
                            first_bytes = f.read(4)
                        if first_bytes[:1] == b'>':
                            print(f"  Valid FASTA detected")
                        elif first_bytes[:2] == b'\x1f\x8b':
                            print(f"  Gzip-compressed file detected")
                        else:
                            print(f"  WARNING: doesn't look like FASTA "
                                  f"(starts with: {first_bytes})")
                else:
                    print(f"  Download failed")
            else:
                print(f"  Could not extract file ID from: {ref_url[:80]}")
        else:
            print("\nNo reference link provided (skipping).")

        # --- Summary ---
        print(f"\n{'='*50}")
        print("DOWNLOAD SUMMARY")
        print(f"{'='*50}")
        if os.path.exists(bam_dir):
            bams = [f for f in os.listdir(bam_dir) if f.endswith('.bam')]
            print(f"  BAM files in {bam_dir}: {len(bams)}")
            for b in sorted(bams):
                print(f"    - {b}")
        if os.path.exists(ref_dir):
            refs = os.listdir(ref_dir)
            print(f"  Reference files in {ref_dir}: {len(refs)}")
            for r in sorted(refs):
                print(f"    - {r}")

    btn.disabled = False
    btn.description = 'Download All'

# --- Layout ---
dl_button = widgets.Button(
    description='Download All',
    button_style='primary',
    icon='download',
    layout=widgets.Layout(width='180px', height='36px')
)
dl_button.on_click(download_files)

display(widgets.VBox([
    widgets.HTML('<b>BAM Files</b> (one link per line):'),
    bam_links,
    widgets.HTML('<br><b>Reference FASTA</b> (single link):'),
    ref_link,
    widgets.HTML('<br>'),
    dl_button,
    widgets.HTML('<br><b>Download Progress:</b>'),
    download_output
]))

## 3. Validate BAM Files & References

Run this cell to check for common issues **before** running MetaPop:
1. **Integrity check** — verifies BAM files aren't corrupted
2. **Re-indexing** — creates fresh `.bai` index files (eliminates stale index issues)
3. **Header vs reference comparison** — checks that contig names in your BAM headers match your reference FASTAs

If you see **"ZERO contig names match"**, that means the BAM files were aligned against different reference sequences than the FASTAs you're providing. You'll need the exact reference used during alignment.

In [ ]:
# === BAM File Validation & Diagnostics ===
# This cell checks for common issues that cause MetaPop to produce empty results.

import os
import subprocess
import gzip

# --- Configuration ---
# These should match the paths used in the cells above
bam_dir = '/content/local_bams'
ref_dir = '/content/local_refs'

# -------------------------------------------------------
# 1. BAM INTEGRITY CHECK
# -------------------------------------------------------
print("=" * 60)
print("1. BAM INTEGRITY CHECK (samtools quickcheck)")
print("=" * 60)

bam_files = sorted([f for f in os.listdir(bam_dir) if f.endswith('.bam')])
print(f"Found {len(bam_files)} BAM file(s)\n")

for f in bam_files:
    bam_path = os.path.join(bam_dir, f)
    ret = subprocess.run(['samtools', 'quickcheck', bam_path],
                         capture_output=True, text=True)
    status = "PASS" if ret.returncode == 0 else "FAIL"
    print(f"  [{status}] {f}")
    if ret.returncode != 0:
        print(f"         Error: {ret.stderr.strip()}")

# -------------------------------------------------------
# 1b. CHECK FOR NAMING ISSUES
# -------------------------------------------------------
print(f"\n{'=' * 60}")
print("1b. BAM FILE NAMING CHECK")
print("=" * 60)

has_naming_issues = False
for f in bam_files:
    issues = []
    if f.startswith('Copy of '):
        issues.append("starts with 'Copy of ' (Google Drive duplicate)")
    if ' ' in f:
        issues.append("contains spaces")
    if issues:
        has_naming_issues = True
        print(f"  WARNING: {f}")
        for issue in issues:
            print(f"           -> {issue}")

if has_naming_issues:
    print("\n  Renaming BAM files to clean names...")
    new_bam_files = []
    for f in bam_files:
        old_path = os.path.join(bam_dir, f)
        clean_name = f
        if clean_name.startswith('Copy of '):
            clean_name = clean_name[len('Copy of '):]
        clean_name = clean_name.replace(' ', '_')
        new_path = os.path.join(bam_dir, clean_name)
        if old_path != new_path:
            # Also rename the index file
            for idx_ext in ['.bai']:
                old_idx = old_path + idx_ext
                new_idx = new_path + idx_ext
                if os.path.exists(old_idx):
                    os.rename(old_idx, new_idx)
            os.rename(old_path, new_path)
            print(f"    {f}  ->  {clean_name}")
        new_bam_files.append(clean_name)
    bam_files = new_bam_files
    print("  Done! Files renamed.")
else:
    print("  All filenames look clean.")

# -------------------------------------------------------
# 2. RE-INDEX ALL BAM FILES (fresh .bai files)
# -------------------------------------------------------
print(f"\n{'=' * 60}")
print("2. RE-INDEXING ALL BAM FILES (fresh .bai files)")
print("=" * 60)

for f in bam_files:
    bam_path = os.path.join(bam_dir, f)

    # Remove old indexes
    for idx_path in [bam_path + '.bai', bam_path.replace('.bam', '.bai')]:
        if os.path.exists(idx_path):
            os.remove(idx_path)

    # Check if coordinate-sorted
    ret = subprocess.run(['samtools', 'view', '-H', bam_path],
                         capture_output=True, text=True)
    if 'SO:coordinate' not in ret.stdout:
        print(f"  WARNING: {f} not coordinate-sorted — sorting now...")
        sorted_path = bam_path + '.sorting.tmp.bam'
        subprocess.run(['samtools', 'sort', '-o', sorted_path, bam_path])
        os.replace(sorted_path, bam_path)

    # Create fresh index
    ret = subprocess.run(['samtools', 'index', bam_path],
                         capture_output=True, text=True)
    if ret.returncode == 0:
        print(f"  Indexed: {f}")
    else:
        print(f"  FAILED:  {f} — {ret.stderr.strip()}")

# -------------------------------------------------------
# 3. EXTRACT CONTIG NAMES FROM BAM HEADERS
# -------------------------------------------------------
print(f"\n{'=' * 60}")
print("3. BAM HEADER vs REFERENCE FASTA COMPARISON")
print("=" * 60)

all_bam_contigs = set()

for f in bam_files:
    bam_path = os.path.join(bam_dir, f)
    ret = subprocess.run(['samtools', 'view', '-H', bam_path],
                         capture_output=True, text=True)
    contigs = set()
    for line in ret.stdout.split('\n'):
        if line.startswith('@SQ'):
            for part in line.split('\t'):
                if part.startswith('SN:'):
                    contigs.add(part[3:])
    all_bam_contigs.update(contigs)
    print(f"\n  {f}: {len(contigs)} reference contig(s) in header")
    for c in sorted(contigs)[:5]:
        print(f"    - {c}")
    if len(contigs) > 5:
        print(f"    ... and {len(contigs) - 5} more")

# -------------------------------------------------------
# 4. EXTRACT CONTIG NAMES FROM REFERENCE FASTAs
# -------------------------------------------------------
ref_contigs = set()

def is_gzipped(filepath):
    """Check if a file is gzip-compressed by reading magic bytes."""
    with open(filepath, 'rb') as f:
        return f.read(2) == b'\x1f\x8b'

if os.path.exists(ref_dir):
    ref_files = sorted([f for f in os.listdir(ref_dir)
                        if f.endswith(('.fa', '.fasta', '.fna',
                                       '.fa.gz', '.fasta.gz', '.fna.gz'))])
    print(f"\nReference directory: {len(ref_files)} FASTA file(s)")

    if len(ref_files) == 0:
        # Maybe files have unusual extensions — list everything
        all_files = os.listdir(ref_dir)
        print(f"  No standard FASTA files found. All files in directory:")
        for af in sorted(all_files):
            fpath = os.path.join(ref_dir, af)
            fsize = os.path.getsize(fpath)
            gz = " [gzip-compressed]" if is_gzipped(fpath) else ""
            print(f"    {af} ({fsize:,} bytes){gz}")

    for f in ref_files:
        ref_path = os.path.join(ref_dir, f)
        file_contigs = set()

        # Auto-detect gzip compression
        compressed = is_gzipped(ref_path)
        opener = gzip.open if compressed else open
        mode = 'rt' if compressed else 'r'

        if compressed:
            print(f"  {f}: [gzip-compressed] ", end="")

        try:
            with opener(ref_path, mode) as fh:
                for line in fh:
                    if line.startswith('>'):
                        first_word = line[1:].strip().split()[0]
                        file_contigs.add(first_word)
        except Exception as e:
            print(f"  ERROR reading {f}: {e}")
            # Try binary sniff to identify file type
            with open(ref_path, 'rb') as bf:
                magic = bf.read(4)
            print(f"    File magic bytes: {magic.hex()} ({magic})")
            continue

        ref_contigs.update(file_contigs)
        print(f"  {f}: {len(file_contigs)} contig(s)")
        for c in sorted(file_contigs)[:3]:
            print(f"    - {c}")
        if len(file_contigs) > 3:
            print(f"    ... and {len(file_contigs) - 3} more")
else:
    print(f"\n  WARNING: Reference directory not found: {ref_dir}")
    print(f"  Make sure you ran the download cell above first.")

# -------------------------------------------------------
# 5. DIAGNOSIS
# -------------------------------------------------------
print(f"\n{'=' * 60}")
print("DIAGNOSIS")
print("=" * 60)

if all_bam_contigs and ref_contigs:
    overlap = all_bam_contigs & ref_contigs
    only_in_bam = all_bam_contigs - ref_contigs
    only_in_ref = ref_contigs - all_bam_contigs

    print(f"\n  Unique contigs in BAM headers:  {len(all_bam_contigs)}")
    print(f"  Unique contigs in references:   {len(ref_contigs)}")
    print(f"  Matching contigs:               {len(overlap)}")

    if len(overlap) == 0:
        print("\n  *** PROBLEM: ZERO contig names match! ***")
        print("  This is why MetaPop produces empty results.")
        print("\n  Example BAM contig names:")
        for c in sorted(all_bam_contigs)[:5]:
            print(f"    BAM: '{c}'")
        print("  Example reference contig names:")
        for c in sorted(ref_contigs)[:5]:
            print(f"    REF: '{c}'")
        print("\n  The BAM files were aligned against different references")
        print("  than the FASTA files you are providing to MetaPop.")
        print("  You need the EXACT reference FASTA used during alignment.")
    elif len(overlap) < len(all_bam_contigs):
        pct = 100 * len(overlap) / len(all_bam_contigs)
        print(f"\n  Partial match: {pct:.1f}% of BAM contigs found in references")
        print(f"  {len(only_in_bam)} BAM contig(s) NOT in references:")
        for c in sorted(only_in_bam)[:5]:
            print(f"    Missing: '{c}'")
    else:
        print("\n  All BAM contigs found in reference files!")
        print("  Header/reference mismatch is NOT the problem.")
        print("  Check MetaPop filter settings (coverage, depth thresholds).")
elif not all_bam_contigs:
    print("\n  No contigs found in BAM headers — files may be empty/unmapped.")
elif not ref_contigs:
    print("\n  No reference FASTAs found — cannot compare.")

print(f"\n{'=' * 60}")
print("VALIDATION COMPLETE")
print("=" * 60)

## 4. Import Libraries and Initialize

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Try to import plotly for interactive plots
try:
    import plotly.express as px
    import plotly.graph_objects as go
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False

print("Libraries loaded successfully!")
print(f"Interactive plots (Plotly): {'Available' if PLOTLY_AVAILABLE else 'Not available'}")

## 5. Configure Pipeline Parameters

Use the interactive widgets below to configure your analysis parameters.

In [ ]:
# File path inputs with live validation and dark text
import ipywidgets as widgets
from IPython.display import display, HTML

# Add CSS to make widget text darker and more readable
display(HTML('''
<style>
.jupyter-widgets .widget-text input {
    color: #000000 !important;
    font-weight: 500;
}
.jupyter-widgets .widget-label {
    color: #000000 !important;
    font-weight: bold;
}
.jupyter-widgets .widget-inline-hbox .widget-label,
.jupyter-widgets .widget-inline-vbox .widget-label {
    color: #000000 !important;
    font-weight: bold;
}
</style>
'''))

print("=== File Paths ===")
print("Configure your input/output directories below.")
print()

input_dir = widgets.Text(
    value='/content/local_bams',
    description='BAM Directory:',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='90%'),
    continuous_update=True
)

reference = widgets.Text(
    value='/content/local_refs',
    description='Reference Directory:',
    placeholder='Directory containing reference FASTA files',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='90%'),
    continuous_update=True
)

genes = widgets.Text(
    value='',
    description='Genes File (optional):',
    placeholder='Leave empty to auto-generate with Prodigal',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='90%'),
    continuous_update=True
)

norm_file = widgets.Text(
    value='',
    description='Normalization File:',
    placeholder='Leave empty to auto-generate from BAM read counts',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='90%'),
    continuous_update=True
)

output_dir = widgets.Text(
    value='/content/drive/MyDrive/metapop_results',
    description='Output Directory:',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='90%'),
    continuous_update=True
)

# Status indicator for path validation
path_status = widgets.HTML(value='')

def validate_paths(change=None):
    """Validate paths and update status."""
    status_parts = []
    
    # Check BAM directory
    if os.path.exists(input_dir.value):
        bam_files = [f for f in os.listdir(input_dir.value) if f.endswith('.bam')]
        status_parts.append(f"✅ BAM directory: {len(bam_files)} BAM file(s) found")
    else:
        status_parts.append(f"❌ BAM directory not found")
    
    # Check reference directory
    if os.path.exists(reference.value):
        if os.path.isdir(reference.value):
            ref_files = [f for f in os.listdir(reference.value) 
                        if f.endswith(('.fa', '.fasta', '.fna'))]
            status_parts.append(f"✅ Reference directory: {len(ref_files)} FASTA file(s) found")
        else:
            status_parts.append(f"⚠️ Reference is a file, not directory")
    else:
        status_parts.append(f"❌ Reference directory not found")
    
    # Check output directory
    if os.path.exists(output_dir.value):
        status_parts.append(f"✅ Output directory exists")
    else:
        status_parts.append(f"ℹ️ Output directory will be created")
    
    path_status.value = '<br>'.join(status_parts)

# Connect validation to path changes
input_dir.observe(validate_paths, names='value')
reference.observe(validate_paths, names='value')
output_dir.observe(validate_paths, names='value')

# Initial validation
validate_paths()

# Display widgets
display(widgets.VBox([
    input_dir,
    reference, 
    genes,
    norm_file,
    output_dir,
    widgets.HTML('<hr style="margin: 10px 0;">'),
    widgets.HTML('<b>Path Status:</b>'),
    path_status
]))

In [ ]:
# Filter parameters with live value display and dark text
print("=== Filter Parameters ===")
print("Adjust the sliders below. Values update in real-time.")
print()

min_pct_id = widgets.FloatSlider(
    value=95.0, min=80.0, max=100.0, step=0.5,
    description='Min % Identity:',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='70%'),
    readout=True,
    readout_format='.1f',
    continuous_update=True
)

min_length = widgets.IntSlider(
    value=50, min=25, max=200, step=5,
    description='Min Read Length:',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='70%'),
    readout=True,
    continuous_update=True
)

min_cov = widgets.IntSlider(
    value=20, min=0, max=100, step=5,
    description='Min Coverage (%):',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='70%'),
    readout=True,
    continuous_update=True
)

min_dep = widgets.IntSlider(
    value=10, min=1, max=100, step=1,
    description='Min Depth:',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='70%'),
    readout=True,
    continuous_update=True
)

truncation = widgets.FloatSlider(
    value=10.0, min=0.0, max=49.0, step=1.0,
    description='Truncation (%):',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='70%'),
    readout=True,
    readout_format='.1f',
    continuous_update=True
)

# Live summary of filter settings
filter_summary = widgets.HTML(value='')

def update_filter_summary(change=None):
    """Update the filter summary display."""
    summary = f"""
    <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-top: 10px; color: #000000;">
    <b>Current Filter Settings:</b><br>
    • Reads must have ≥{min_pct_id.value:.1f}% identity to reference<br>
    • Reads must be ≥{min_length.value} bp long<br>
    • Contigs need ≥{min_cov.value}% coverage breadth<br>
    • Positions need ≥{min_dep.value}x read depth<br>
    • Truncate {truncation.value:.0f}% from each contig end
    </div>
    """
    filter_summary.value = summary

# Connect summary update to all sliders
for slider in [min_pct_id, min_length, min_cov, min_dep, truncation]:
    slider.observe(update_filter_summary, names='value')

# Initial summary
update_filter_summary()

# Display with organized layout
display(widgets.VBox([
    min_pct_id,
    min_length,
    min_cov,
    min_dep,
    truncation,
    filter_summary
]))

In [ ]:
# Analysis options with live summary and dark text
print("=== Analysis Options ===")
print("Select which analyses to run and configure threads.")
print()

run_preproc = widgets.Checkbox(
    value=True, 
    description='Run Preprocessing',
    style={'description_width': 'initial', 'description_color': '#000000'},
    indent=False
)

run_microdiv = widgets.Checkbox(
    value=True, 
    description='Run Microdiversity Analysis',
    style={'description_width': 'initial', 'description_color': '#000000'},
    indent=False
)

run_macrodiv = widgets.Checkbox(
    value=True, 
    description='Run Macrodiversity Analysis',
    style={'description_width': 'initial', 'description_color': '#000000'},
    indent=False
)

run_viz = widgets.Checkbox(
    value=True, 
    description='Generate Visualizations',
    style={'description_width': 'initial', 'description_color': '#000000'},
    indent=False
)

threads = widgets.IntSlider(
    value=4, min=1, max=16, step=1,
    description='Threads:',
    style={'description_width': '150px', 'description_color': '#000000'},
    layout=widgets.Layout(width='50%'),
    readout=True,
    continuous_update=True
)

# Live summary of what will run
analysis_summary = widgets.HTML(value='')

def update_analysis_summary(change=None):
    """Update the analysis summary."""
    analyses = []
    if run_preproc.value:
        analyses.append("✅ Preprocessing (filter reads, compute coverage)")
    else:
        analyses.append("⏭️ Skip preprocessing")
    
    if run_microdiv.value:
        analyses.append("✅ Microdiversity (SNPs, pi, theta, Tajima's D, pN/pS)")
    else:
        analyses.append("⏭️ Skip microdiversity")
    
    if run_macrodiv.value:
        analyses.append("✅ Macrodiversity (richness, Shannon's H, beta diversity)")
    else:
        analyses.append("⏭️ Skip macrodiversity")
    
    if run_viz.value:
        analyses.append("✅ Visualizations (plots and figures)")
    else:
        analyses.append("⏭️ Skip visualizations")
    
    summary = f"""
    <div style="background-color: #e8f4e8; padding: 10px; border-radius: 5px; margin-top: 10px; color: #000000;">
    <b>Pipeline Configuration:</b><br>
    {'<br>'.join(analyses)}<br>
    <br>
    <b>Using {threads.value} thread(s)</b> for parallel processing
    </div>
    """
    analysis_summary.value = summary

# Connect to all checkboxes and threads
for widget in [run_preproc, run_microdiv, run_macrodiv, run_viz, threads]:
    widget.observe(update_analysis_summary, names='value')

# Initial summary
update_analysis_summary()

# Organize checkboxes in columns
checkbox_box = widgets.HBox([
    widgets.VBox([run_preproc, run_microdiv]),
    widgets.VBox([run_macrodiv, run_viz])
])

display(widgets.VBox([
    checkbox_box,
    widgets.HTML('<br>'),
    threads,
    analysis_summary
]))

## 6. Run MetaPop Pipeline

In [ ]:
import subprocess
import time
import ipywidgets as widgets
from IPython.display import display, HTML

# --- Confirmation summary (updates live as you change widgets above) ---
confirm_html = widgets.HTML()

def update_confirmation(change=None):
    """Build a live summary of all current settings."""
    analyses = []
    if run_preproc.value:  analyses.append("Preprocessing")
    if run_microdiv.value: analyses.append("Microdiversity")
    if run_macrodiv.value: analyses.append("Macrodiversity")
    if run_viz.value:      analyses.append("Visualizations")

    confirm_html.value = f"""
    <div style="background:#fffde7; border:2px solid #f9a825; border-radius:8px;
                padding:14px; margin:10px 0; color:#000;">
      <h3 style="margin-top:0; color:#e65100;">Review Settings Before Running</h3>
      <table style="color:#000; font-size:14px; border-collapse:collapse;">
        <tr><td style="padding:3px 12px 3px 0;"><b>BAM Directory:</b></td>
            <td><code>{input_dir.value}</code></td></tr>
        <tr><td style="padding:3px 12px 3px 0;"><b>Reference Dir:</b></td>
            <td><code>{reference.value}</code></td></tr>
        <tr><td style="padding:3px 12px 3px 0;"><b>Genes File:</b></td>
            <td><code>{genes.value or '(auto-generate with Prodigal)'}</code></td></tr>
        <tr><td style="padding:3px 12px 3px 0;"><b>Norm File:</b></td>
            <td><code>{norm_file.value or '(auto-generate from BAM read counts)'}</code></td></tr>
        <tr><td style="padding:3px 12px 3px 0;"><b>Output Dir:</b></td>
            <td><code>{output_dir.value}</code></td></tr>
        <tr><td colspan="2" style="padding:6px 0 3px;"><hr style="margin:0;"></td></tr>
        <tr><td style="padding:3px 12px 3px 0;"><b>Min % Identity:</b></td>
            <td>{min_pct_id.value:.1f}%</td></tr>
        <tr><td style="padding:3px 12px 3px 0;"><b>Min Read Length:</b></td>
            <td>{min_length.value} bp</td></tr>
        <tr><td style="padding:3px 12px 3px 0;"><b>Min Coverage:</b></td>
            <td>{min_cov.value}%</td></tr>
        <tr><td style="padding:3px 12px 3px 0;"><b>Min Depth:</b></td>
            <td>{min_dep.value}x</td></tr>
        <tr><td style="padding:3px 12px 3px 0;"><b>Truncation:</b></td>
            <td>{truncation.value:.0f}%</td></tr>
        <tr><td colspan="2" style="padding:6px 0 3px;"><hr style="margin:0;"></td></tr>
        <tr><td style="padding:3px 12px 3px 0;"><b>Analyses:</b></td>
            <td>{', '.join(analyses) if analyses else '<span style="color:red;">None selected!</span>'}</td></tr>
        <tr><td style="padding:3px 12px 3px 0;"><b>Threads:</b></td>
            <td>{threads.value}</td></tr>
      </table>
    </div>
    """

# Connect all widgets to live-update the confirmation
for w in [input_dir, reference, genes, norm_file, output_dir,
          min_pct_id, min_length, min_cov, min_dep, truncation,
          run_preproc, run_microdiv, run_macrodiv, run_viz, threads]:
    w.observe(update_confirmation, names='value')

update_confirmation()

# --- Progress output area (separate from button so it doesn't disappear) ---
progress_output = widgets.Output(layout={'border': '1px solid #ccc',
                                         'min_height': '100px',
                                         'max_height': '500px',
                                         'overflow_y': 'auto',
                                         'padding': '8px'})
status_html = widgets.HTML(value='<i>Ready. Click "Run MetaPop" to start.</i>')

def run_metapop(btn):
    """Run the MetaPop pipeline with configured parameters."""
    btn.disabled = True
    btn.description = 'Running...'
    btn.icon = 'spinner'
    progress_output.clear_output()
    status_html.value = '<b style="color:#1565c0;">Running MetaPop pipeline...</b>'

    with progress_output:
        # Validate inputs
        if not os.path.exists(input_dir.value):
            print(f"Error: BAM directory not found: {input_dir.value}")
            status_html.value = '<b style="color:red;">Error: BAM directory not found</b>'
            btn.disabled = False
            btn.description = 'Run MetaPop'
            btn.icon = 'play'
            return

        ref_path = reference.value
        if os.path.isfile(ref_path):
            print(f"Note: Reference is a file — using parent directory.")
            ref_path = os.path.dirname(ref_path)

        if not os.path.exists(ref_path):
            print(f"Error: Reference directory not found: {ref_path}")
            status_html.value = '<b style="color:red;">Error: Reference directory not found</b>'
            btn.disabled = False
            btn.description = 'Run MetaPop'
            btn.icon = 'play'
            return

        os.makedirs(output_dir.value, exist_ok=True)

        # Build command
        cmd = [
            'metapop',
            '--input_samples', input_dir.value,
            '--reference', ref_path,
            '--output', output_dir.value,
            '--id_min', str(min_pct_id.value),
            '--min_len', str(min_length.value),
            '--min_cov', str(min_cov.value),
            '--min_dep', str(min_dep.value),
            '--trunc', str(truncation.value),
            '--threads', str(threads.value),
        ]

        if genes.value:
            cmd.extend(['--genes', genes.value])
        if norm_file.value:
            cmd.extend(['--norm', norm_file.value])
        if not run_microdiv.value:
            cmd.append('--no_micro')
        if not run_macrodiv.value:
            cmd.append('--no_macro')
        if not run_viz.value:
            cmd.append('--no_viz')

        print(f"Command: {' '.join(cmd)}")
        print("=" * 50)
        start_time = time.time()

        try:
            process = subprocess.Popen(
                cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                bufsize=1
            )

            for line in process.stdout:
                print(line, end='')

            process.wait()
            elapsed = time.time() - start_time

            if process.returncode == 0:
                print(f"\n{'=' * 50}")
                print(f"Pipeline completed in {elapsed:.0f}s")
                print(f"Results saved to: {output_dir.value}")
                status_html.value = (
                    f'<b style="color:green;">Complete!</b> '
                    f'Finished in {elapsed:.0f}s. Results in <code>{output_dir.value}</code>'
                )
            else:
                print(f"\nMetaPop exited with code: {process.returncode}")
                status_html.value = (
                    f'<b style="color:red;">Failed</b> (exit code {process.returncode}). '
                    f'See output above.'
                )

        except FileNotFoundError:
            print("Error: MetaPop command not found.")
            print("Try running: !pip install -e /content/metapop")
            status_html.value = '<b style="color:red;">Error: metapop command not found</b>'
        except Exception as e:
            print(f"Error: {e}")
            import traceback
            traceback.print_exc()
            status_html.value = f'<b style="color:red;">Error: {e}</b>'

    btn.disabled = False
    btn.description = 'Run MetaPop'
    btn.icon = 'play'

# --- Layout ---
run_button = widgets.Button(
    description='Run MetaPop',
    button_style='success',
    tooltip='Click to start the pipeline with the settings shown above',
    icon='play',
    layout=widgets.Layout(width='200px', height='40px')
)
run_button.on_click(run_metapop)

display(widgets.VBox([
    confirm_html,
    widgets.HBox([run_button, widgets.HTML('&nbsp;&nbsp;'), status_html]),
    widgets.HTML('<br><b>Pipeline Output:</b>'),
    progress_output
]))

## 7. View Results

After the pipeline completes, use the cells below to explore results interactively.

In [ ]:
def load_results():
    """Load and display MetaPop results."""
    results_path = os.path.join(output_dir.value, 'MetaPop')
    
    if not os.path.exists(results_path):
        print(f"Results not found at: {results_path}")
        print("Please run the pipeline first.")
        return None
    
    results = {}
    
    # Load alpha diversity
    alpha_file = os.path.join(results_path, '11.Macrodiversity', 'Alpha_diversity_stats.tsv')
    if os.path.exists(alpha_file):
        results['alpha'] = pd.read_csv(alpha_file, sep='\t', index_col=0)
        print(f"Loaded alpha diversity: {len(results['alpha'])} samples")
    
    # Load gene microdiversity
    gene_micro_file = os.path.join(results_path, '10.Microdiversity', 'global_gene_microdiversity.tsv')
    if os.path.exists(gene_micro_file):
        results['gene_micro'] = pd.read_csv(gene_micro_file, sep='\t')
        print(f"Loaded gene microdiversity: {len(results['gene_micro'])} genes")
    
    # Load abundance matrix
    abundance_file = os.path.join(results_path, '11.Macrodiversity', 'normalized_abundances_table.tsv')
    if os.path.exists(abundance_file):
        results['abundance'] = pd.read_csv(abundance_file, sep='\t', index_col=0)
        print(f"Loaded abundance matrix: {results['abundance'].shape}")
    
    return results

results = load_results()

In [ ]:
# Display alpha diversity summary
if results and 'alpha' in results:
    print("Alpha Diversity Summary:")
    display(results['alpha'].describe())
    
    if PLOTLY_AVAILABLE:
        # Interactive bar chart
        fig = px.bar(results['alpha'].reset_index(), x='index', y='Richness',
                    title='Species Richness by Sample',
                    labels={'index': 'Sample', 'Richness': 'Species Richness'})
        fig.show()
    else:
        # Static matplotlib plot
        plt.figure(figsize=(10, 5))
        results['alpha']['Richness'].plot(kind='bar')
        plt.title('Species Richness by Sample')
        plt.ylabel('Richness')
        plt.tight_layout()
        plt.show()

In [ ]:
# Display pN/pS distribution
if results and 'gene_micro' in results:
    pnps = results['gene_micro']['pNpS_ratio'].dropna()
    pnps = pnps[~np.isinf(pnps) & (pnps < 10)]  # Filter outliers
    
    print(f"pN/pS Statistics:")
    print(f"  Mean: {pnps.mean():.3f}")
    print(f"  Median: {pnps.median():.3f}")
    print(f"  Genes with pN/pS < 1 (purifying): {(pnps < 1).sum()}")
    print(f"  Genes with pN/pS > 1 (positive): {(pnps > 1).sum()}")
    
    if PLOTLY_AVAILABLE:
        fig = px.histogram(pnps, nbins=50, title='pN/pS Ratio Distribution',
                          labels={'value': 'pN/pS Ratio', 'count': 'Gene Count'})
        fig.add_vline(x=1, line_dash='dash', line_color='red',
                     annotation_text='Neutral (pN/pS=1)')
        fig.show()
    else:
        plt.figure(figsize=(10, 5))
        plt.hist(pnps, bins=50, edgecolor='black')
        plt.axvline(1, color='red', linestyle='--', label='Neutral')
        plt.xlabel('pN/pS Ratio')
        plt.ylabel('Gene Count')
        plt.title('pN/pS Distribution')
        plt.legend()
        plt.tight_layout()
        plt.show()

## 8. Download Results

Download the results as a ZIP file.

In [ ]:
from google.colab import files
import shutil

def download_results():
    """Create ZIP archive and download results."""
    results_path = os.path.join(output_dir.value, 'MetaPop')
    
    if not os.path.exists(results_path):
        print("No results to download.")
        return
    
    zip_path = '/content/metapop_results'
    shutil.make_archive(zip_path, 'zip', results_path)
    
    print(f"Downloading {zip_path}.zip...")
    files.download(f'{zip_path}.zip')

download_button = widgets.Button(
    description='Download Results',
    button_style='info',
    tooltip='Download results as ZIP',
    icon='download'
)
download_button.on_click(lambda b: download_results())

display(download_button)

---

## Need Help?

- **Documentation**: See the MetaPop README for detailed usage instructions
- **Issues**: Report bugs at https://github.com/espickle1/metapop/issues
- **Contact**: James.Chang@bcm.edu